# `server/udp/podCommands.js` - from node.js /JavaScript to Python (object)

In [2]:
# find out where we are on the file directory
import os, sys

In [3]:
print( os.getcwd())
print( os.listdir(os.getcwd()))

/home/topolo/PropD/servetheloop/packetDef
['.ipynb_checkpoints', 'reactGS', 'podCommands.ipynb', 'react-GS_node_to_py_to_YAML.ipynb']


The `reactGS` folder "mimics" the actual `react-groundstation` github repository, only copying the file directory structure, but the source code itself (which is a lot) isn't completely copied over.  I wanted to keep these scripts/notebooks/files built on top of that github repository to be separate from the actual working code.  

In [4]:
wherepodCommandsis = os.getcwd()+'/reactGS/server/udp/' 
print(wherepodCommandsis)

/home/topolo/PropD/servetheloop/packetDef/reactGS/server/udp/


## `node.js`/(JavaScript) to `json`; i.e. 

## `node.js`/(JavaScript) $\to$ `json`    

Make a copy of `server/udp/podCommands.js`.  

In this copy, comment out `var chalk = require('chalk')` (this is the only thing you have to do manually).  

Run this in the directory containing your copy of `podCommands.js`:

```  
node traverse_podCommands.js  
```  

This should generate a `json` file `podCmds_lst.json`

# Available `podCommands` as a Python list; `json` to Python list, i.e. `json` $\to$ Python list

In [7]:
import json

In [10]:
f_podCmds_json = open(wherepodCommandsis+'podCmds_lst.json','rb')
rawjson_podCmds = f_podCmds_json.read()
f_podCmds_json.close()

In [16]:
print(type(rawjson_podCmds))
podCmds_lst=json.loads(rawjson_podCmds)
print(type(podCmds_lst))
print(len(podCmds_lst)) # there are 104 available commands for the pod!

<type 'str'>
<type 'list'>
104


In [17]:
for cmd in podCmds_lst: 
    print cmd

GS_Heartbeat
LGU_PositionChange
LGU_SpeedChange
FCUPod_Off
FCUPod_Stop
FCUPod_PowerLatch
setBrakeDevelopmentMode
FCUBrake_DisableDevelopmentMode
FCUBrake_EnableDevelopmentMode
FCUBrake_MoveMotorRAW
FCUBrake_MoveMotorIBeam
FCUBrake_BeginInit
FCUBrake_MLPSetZeroLeftBrake
FCUBrake_MLPSetZeroRightBrake
FCUBrake_MLPSetSpanLeftBrake
FCUBrake_MLPSetSpanRightBrake
FCUStepper_SetMaxAngularAccel
FCUStepper_SetPicoMetersPerRev
FCUStepper_SetMaxRPM
FCUStepper_SetMicroStepResolution
FCUBrake_RequestDevelopmentMode
FCUStreamingControlStart_AccelCalData
FCUStreamingControlStart_AccelFullData
FCUStreamingControlStop_Accel
FCUStreamingControlStart_Brakes
FCUStreamingControlStart_MotorsRaw
FCUStreamingControlStart_Lasers
FCUStreamingControlStart_ForwardLaser
FCUAccel_FineZero
FCUAccel_AutoZero
FCUContrast_StartStream
FCUContrast_StopStream
PowerAStopAllManualDischarging
PowerBStopAllManualDischarging
PowerAPowerLatch
PowerBPowerLatch
PowerAChargeRelayOff
PowerAChargeRelayOn
PowerAStreamingOff
PowerBStre

## Dirty parsing of `podCommands.js` and the flight control parameters 

In [38]:
f_podCmds = open(wherepodCommandsis+'podCommands.js','rb')
raw_podCmds = f_podCmds.read()
f_podCmds.close()
print(type(raw_podCmds))
print(len(raw_podCmds))

<type 'str'>
19296


In [39]:
print(len(raw_podCmds.split("function ")) ) # 106 commands

# get the index value (e.g. starts at position 22) of where "udp.tx.transmitPodCommand" starts, treating it as a string
whereisudptransmit = [func.find("udp.tx.transmitPodCommand(") for func in raw_podCmds.split("function ")]

# remove -1 values
#whereisudptransmit = filter(lambda x : x != -1, whereisudptransmit)

rawParams=[funcstr[ funcstr.find("(",val)+1:funcstr.find(")",val)]  if val is not -1 else None for funcstr, val in zip(raw_podCmds.split("function "), whereisudptransmit)]

106


In [104]:
# more parsing of this list of strings
cleaningParams = []
for rawparam in rawParams: 
    if rawparam is None:
        cleaningParams.append(None)
    else:
        cleanParam = []
        cleanParam.append( rawparam.split(',')[0].strip("'") )
        for strval in rawparam.split(',')[1:]:
            strval2 = strval.strip()
            try: 
                strval2 = int(strval2,16)
                strval2 = hex(strval2)
            except ValueError:
                strval2
            cleanParam.append(strval2)
        cleaningParams.append(cleanParam)


In [105]:
# get the name of the functions

#[func[:func.find("(")]  
#    if func.find("()") is not -1 else None for func in raw_podCmds.split("function ")];
cmdnameslst = [func[:func.find("(")].strip() for func in raw_podCmds.split("function ")]

In [106]:
podCommandparams = zip(cmdnameslst, cleaningParams)
print(len(podCommandparams))

106


In [107]:
podCommandparams[-2]

('PodSafePowerNodeB',
 ['Power Node B', '0x3000', '0x76543210', '0x0', '0x0', '0x0'])

So the structure of our result is as follows:

Python tuples (each of size 2 for each of the tuples)  
"""   
    ( Name of pod command as a string,  
        Python list [ Subsystem name as a string, paramter1 as a hex value, paramter2 as a hex value, paramter3 as a hex value, paramter4 as a hex value] )  
"""  

Notice that in the original code, there's some TO DO's still left (eek!) so that those `udp.tx.transmitPodCommand` is commented out or left as TODO, and some are dependent upon arguments in the function (and thus will change, the parameter is a variable).  

In [114]:
podCommandparams[:10]

[('const bin = require', None),
 ('', None),
 ('GS_Heartbeat', ['Flight Control', '0x400', '0x0', '0x0', '0x0', '0x0']),
 ('LGU_PositionChange', ['????', '0x0', '0x0', '0x0', '0x0', '0x0']),
 ('LGU_SpeedChange', ['????', '0x0', '0x0', '0x0', '0x0', '0x0']),
 ('setBrakeDevelopmentMode', None),
 ('FCUPod_Off',
  ['Flight Control', '0x3000', '0x76543210', '0x0', '0x0', '0x0']),
 ('FCUPod_Stop', ['Flight Control', '0x1', '0x1234abcd', '0x0', '0x0', '0x0']),
 ('FCUPod_PowerLatch',
  ['Flight Control', '0x3030', '0xabcd1245', 'data.powerNode', '0x0', '0x0']),
 ('FCUBrake_DisableDevelopmentMode',
  ['Flight Control', '0x1400', '0x0', '0x0', '0x0', '0x0'])]

In [109]:
try:
    import CPickle as pickle
except ImportError:
    import pickle

In [110]:
podCommandparamsfile = open("podCommandparams.pkl",'wb')
pickle.dump( podCommandparams , podCommandparamsfile )
podCommandparamsfile.close()

In [111]:
# open up a pickle file like so:
podCommandparamsfile_recover = open("podCommandparams.pkl",'rb')
podCommandparams_recover = pickle.load(podCommandparamsfile_recover)
podCommandparamsfile_recover.close()

In [113]:
podCommandparams_recover[:10]

[('const bin = require', None),
 ('', None),
 ('GS_Heartbeat', ['Flight Control', '0x400', '0x0', '0x0', '0x0', '0x0']),
 ('LGU_PositionChange', ['????', '0x0', '0x0', '0x0', '0x0', '0x0']),
 ('LGU_SpeedChange', ['????', '0x0', '0x0', '0x0', '0x0', '0x0']),
 ('setBrakeDevelopmentMode', None),
 ('FCUPod_Off',
  ['Flight Control', '0x3000', '0x76543210', '0x0', '0x0', '0x0']),
 ('FCUPod_Stop', ['Flight Control', '0x1', '0x1234abcd', '0x0', '0x0', '0x0']),
 ('FCUPod_PowerLatch',
  ['Flight Control', '0x3030', '0xabcd1245', 'data.powerNode', '0x0', '0x0']),
 ('FCUBrake_DisableDevelopmentMode',
  ['Flight Control', '0x1400', '0x0', '0x0', '0x0', '0x0'])]